In [1]:
from helpers import *
import fnmatch

from shapely import Point, LineString
import re

### regex example

In [2]:
# Define the string
# example of using regex to find the digits in a string 

string = "Vertex_2_Zcoordinate"

# Define the regex pattern to match digits
pattern = re.compile(r'\d+')

# Find all matches of the pattern in the string
matches = pattern.findall(string)
matches

['2']

## start

In [3]:
e = EneryPlusCase("test01")

In [4]:
EAST_COORDS = [(10,0), (10,7), (5,7), (5,0)]
WEST_COORDS = [(5,0), (5,7), (0,7), (0,0)]
NORTH_COORDS = [(10,7), (10,11), (0,11), (0,7)]

e.idf.add_block(
      name='North',
      coordinates=NORTH_COORDS,
      height=3
    )

In [5]:
def get_walls(zone_surfaces):
    walls = []
    num_vertices = 4 # TODO always check this.. 
    for surface in zone_surfaces:
        valid_fieldnames = fnmatch.filter(surface.fieldnames, f"Vertex_[0-{num_vertices}]_Zcoordinate")

        valid_values = []
        for fieldname in valid_fieldnames:  
                valid_values.append(surface[fieldname])

        if len(set(valid_values)) == 2:
            walls.append(surface)
        
    return walls


In [6]:
current_zone = "Block North Storey 0"
all_surfaces = e.idf.idfobjects["BUILDINGSURFACE:DETAILED"]
zone_surfaces = [a for a in all_surfaces if a.Zone_Name==current_zone]

In [7]:
walls = get_walls(zone_surfaces)
wall = walls[0]
wall


BUILDINGSURFACE:DETAILED,
    Block North Storey 0 Wall 0001,    !- Name
    wall,                     !- Surface Type
    ,                         !- Construction Name
    Block North Storey 0,     !- Zone Name
    ,                         !- Space Name
    ,                         !- Outside Boundary Condition
    ,                         !- Outside Boundary Condition Object
    SunExposed,               !- Sun Exposure
    WindExposed,              !- Wind Exposure
    autocalculate,            !- View Factor to Ground
    autocalculate,            !- Number of Vertices
    10,                       !- Vertex 1 Xcoordinate
    7,                        !- Vertex 1 Ycoordinate
    3,                        !- Vertex 1 Zcoordinate
    10,                       !- Vertex 2 Xcoordinate
    7,                        !- Vertex 2 Ycoordinate
    0,                        !- Vertex 2 Zcoordinate
    10,                       !- Vertex 3 Xcoordinate
    11,                       !- Vert

In [8]:
z_coords = fnmatch.filter(wall.fieldnames, "Vertex_[0-4]_Zcoordinate")
z_coords

['Vertex_1_Zcoordinate',
 'Vertex_2_Zcoordinate',
 'Vertex_3_Zcoordinate',
 'Vertex_4_Zcoordinate']

In [9]:
# this is carried out for a specific wall 

z_coords = fnmatch.filter(wall.fieldnames, "Vertex_[0-4]_Zcoordinate")


vertices = []
for fieldname in z_coords:
    if wall[fieldname] == 0:
        matches = pattern.findall(fieldname)
        # TODO some tests needed here.. 
        x_field = fnmatch.filter(wall.fieldnames, f"Vertex_{matches[0]}_Xcoordinate")[0]
        x_val = wall[x_field]

        y_field =  fnmatch.filter(wall.fieldnames, f"Vertex_{matches[0]}_Ycoordinate")[0]
        y_val = wall[y_field]
        vertices.append([x_val, y_val])
        


ic(vertices) 
line = LineString(vertices)     
print(line)

ic| vertices: [[10.0, 7.0], [10.0, 11.0]]


LINESTRING (10 7, 10 11)


In [10]:
a = Point(2,3)

### selecting

In [11]:
from helper_classes import *
from geometry_parser import *

In [12]:
g = GeometryParser(e.idf)

In [13]:
g.get_zones()


In [14]:
g.plot_zones()

In [15]:
g.zones

[Zone(Block North Storey 0)]

In [16]:
len(g.zones[0].walls)

4

In [17]:
g.zones[0].walls[0].line.coords

In [18]:
def plot_LineString(line:sp.LineString, color="yellow", label=None):
    x, y = points_to_plot(line.coords)
    trace = go.Scatter(x=x, y=y, mode='markers+lines', marker=dict(color=color),  line=dict(color=color), name=label)
    return trace

In [19]:
traces = []
for zone in g.zones:
    for wall in zone.walls:
        trace = plot_LineString(wall.line)
        traces.append(trace)

In [20]:
fig = go.Figure()
for t in traces:
    fig.add_trace(t)
fig.show()


In [21]:
# plot this .. 